### To obtain features from Butterworth filter
#### This method contains 3 classes
1. `Sequence`: Convert multiple line fasta sequence to single line fasta sequence.

2. `signal_pseaac`: The brain of the program.
    a. Converts the amino acid sequence signals.
    b. Generates Butterworths low and high pass filter coefficent using [`butter`][1].
    c. Capture low pass freqency and high pass frequency.
    d. Calculates the features p1,p2,p3,p4,p5.
    
3. `yL`: my code for both [`ifiltic`][2] + [`lfilter`][3].

    [Algorithm][4]: $$y_{L}(n)=-\sum\limits_{k=2}^{26} a(k)y_{L}(n-k+1)+\sum\limits_{k=1}^{26} b(k)x(n-k+1)$$
    
[1]:https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html#scipy.signal.butter
[3]:https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html#scipy.signal.lfilter
[2]:https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfiltic.html#scipy.signal.lfiltic
[4]:https://pubmed.ncbi.nlm.nih.gov/13678304/

In [171]:
import numpy as np
from scipy import stats
import statistics as stat
import scipy.signal


#scale 14 @ 
def properties(a):# Standard the physiochemical properties
    temp,r2={},{}
    with open('7_98_hydrophobicity.csv','r') as inpt:
        c=0
        for i in inpt:
            j=i.rstrip().split(',')
            if c==0:
                pass
            else:
                temp[j[0]]=float(j[a])
            c+=1
    r1=stats.zscore(np.array(list(temp.values())))
    for ii,jj in zip(temp.keys(),r1):
        r2[ii]=jj
    return r2
  
no_prop=[r for r in range(1,105)]# 1: Hydrophobicity, 2: Hydrophilicity, 3: mass, 4: pk1, 5:pk2, 6:pi, 20: 14 scale, 60: Tanford
prop={}
for t1 in no_prop:
    prop[t1]=properties(t1)

class Sequence:
    def __init__(self,filename):
        self.data=[]
        if type(filename)==str:
            with open(filename,'r') as inpt:
                for each in inpt:
                    self.data.append(each.rstrip())
        else:
            self.data=filename
                       
    def output(self):
        a,s,l=[],[],[]
        unusual=0
        for ele in self.ml_sl():
            if ele.startswith('>'):
                a.append(ele.rstrip())
            else:
                if 'X' in ele:
                    ele=ele.replace('X','A') # replacing X with A amino acid as in Trembl(Ala (A) 9.26 Leu (L) 9.91)
                    unusual=+1
                l.append(len(ele.rstrip()))
                s.append(ele.rstrip().upper())
        print('The length of the smallest sequence:',min(l))
        print('Sequence with "X" present:',unusual)
        return a,s,l
             
    def ml_sl(self):
        acc_seq=[]
        for k in range(len(self.data)):
            if self.data[k].startswith('>'):
                acc_seq.append(self.data[k]+'\n')
                join_=0
                for l in range(k+1,len(self.data)):
                    if self.data[l].startswith('>') == False:
                        join_+=1
                    else:
                        break
                acc_seq.append(''.join(self.data[k+1:k+1+join_])+'\n')
        return acc_seq
    
class signal_pseaac:
    def __init__(self,k):
        self.k=k
        # Assigning numerical values to each amino acid from 1 to 20
        aa='acdefghiklmnpqrstvwy'
#         self.aa_val={aa[i].upper():i+1 for i in range(len(aa))}
        self.aa_val=prop[1]
        
        # Butterworth filter Yu et.al(2003) wrong values in the paper
        
#         a_i='1.00 5.00 15.11 32.49 54.80 75.51 87.53 86.78 74.49 55.78 36.63 21.16 10.77 4.82 1.90 0.65 0.20 0.05 1.13e-2 2.11e-3 3.27e-4 4.08e-5 3.95e-6 2.79e-7 1.27e-8 2.83e-10'
#         b_i='1.68e-5 4.21e-4 5.05e-3 3.87e-2 0.21 0.89 2.98 8.09 18.2 34.38 55.01 75.01 87.51 87.51 75.01 55.01 34.38 18.20 8.09 2.98 0.89 0.21 0.04 5.05e-3 4.21e-4 1.68e-5'
#         c_i='1.00 −5.00 15.11 −32.49 54.80 −75.51 87.53 −86.78 74.49 −55.78 36.63 −21.16 10.77 −4.82 1.90 −0.65 0.20 −0.05 0.01 −2.11e-3 3.27e-4 −4.08e-5 3.95e-6 −2.79e-7 1.27e-8 −2.83e-10'.replace('−','-')
#         d_i='1.68e-5 −4.21e-4 5.05e-3 −3.87e-2 0.21 −0.89 2.98 −8.09 18.20 −34.38 55.01 −75.01 87.51 −87.51 75.01 −55.01 34.38 −18.20 8.09 −2.98 0.89 −0.21 3.87e-2 −5.05e-3 4.21e-4 −1.68e-5'.replace('−','-')
        # Low-Pass Butterworth filter
        b_i,a_i=scipy.signal.butter(25,0.5,btype='low')
        d_i,c_i =scipy.signal.butter(25,0.5,btype='high')
        self.a_i={i+1:float(a_i[i]) for i in range(len(a_i))}
        self.b_i={i+1:float(b_i[i]) for i in range(len(b_i))}
        # High-Pass Butterworth filter
        self.c_i={i+1:float(c_i[i]) for i in range(len(c_i))}
        self.d_i={i+1:float(d_i[i]) for i in range(len(d_i))}
        self.w={1:0.5,2:0.015,3:0.005,4:0.5,5:0.5}
        
    def signal_otpt(self,ac,se):
        self.ac=ac
        self.se=se
        self.all_p=[]
        for i in range(len(self.se)):
            self.signal=[self.aa_val[j] for j in self.se[i]]
            self.p1,self.p2,self.p3=stat.mean(self.signal),stat.pstdev(self.signal),stat.pvariance(self.signal)
            temp=[self.p1,self.p2,self.p3]
            low_filt_otpt,high_filt_otpt=[],[]
            intial=yL(self.k,self.signal)
            for res in range(len(self.signal)):
                low_filt_otpt.append(intial.total(res+1,self.a_i,self.b_i))

            intial=yL(self.k,self.signal)
            for res in range(len(self.signal)):
                high_filt_otpt.append(intial.total(res+1,self.c_i,self.d_i))
            self.p4=self.corrcoef(low_filt_otpt)
            self.p5=self.corrcoef(high_filt_otpt)
            temp=temp+[self.p4,self.p5]
            all_val=[self.ac[i][1:]]+self.pseaac(temp,self.se[i])
            self.all_p.append(all_val)
            temp=[]
        return self.all_p
    
    def pseaac(self,param,each_seq):
        p=[]
        deno=1+sum([self.w[i+1]*param[i] for i in range(5)])
        for u in range(1,self.k):
            if u>=1 and u<=20:
                num=each_seq.count(list(self.aa_val.keys())[u-1])/len(each_seq)
                p.append(num/deno)
            elif u>=21 and u<=self.k:
                num1=self.w[u-20]*param[u-21]
                p.append(num1/deno)
        return p
        
    
    def corrcoef(self,output):
        avg_output=stat.mean(output)
        cov=(sum((self.signal[ind]-self.p1)*(output[ind]-avg_output) for ind in range(len(self.signal))))/(len(self.signal)-1)
        return cov/(stat.stdev(self.signal)*stat.stdev(output))
    
class yL:
    def __init__(self,k,signal):
        self.k=k
        self.ylow={-i:0 for i in range(self.k-1)}
        self.signal=signal
        
    def ay(self,a_tab):
        all_a=[]
        for each in range(2,self.k+1):
            part1=a_tab[each]
            part2=self.ylow[self.n-each+1]
            all_a.append(part1*part2)
        return sum(all_a)
            
    def bx(self,b_tab):
        all_b=[]
        for each in range(1,self.k+1):
            part1=b_tab[each]
            if self.n-each+1<=0:
                part2=stat.mean(self.signal) # avg for any value of x less the 1
            else:
                part2=self.signal[self.n-each+1-1]
            all_b.append(part1*part2)
        return sum(all_b)
    
    def total(self,n,a_tab,b_tab):
        self.n=n
        tot=-self.ay(a_tab)+self.bx(b_tab)
        self.ylow[self.n]=tot
        return tot
           
acc,seq,min_len=Sequence('gh_lyso_c_g_ch.txt').output()
a=signal_pseaac(26)
b=a.signal_otpt(acc,seq)

The length of the smallest sequence: 125
Sequence with "X" present: 0


In [172]:
import pandas as pd
head=[f'x{i}' for i in range(26)]
df=pd.DataFrame(b,columns=head)
df.to_csv('lyso_signal_butter_hydro.csv',sep='\t',index=False)

In [174]:
xx=df.iloc[:,1:].values
xx

array([[ 0.10621894,  0.01416252,  0.06019073, ...,  0.00491326,
         0.05193201,  0.02721879],
       [ 0.08843005,  0.03275187,  0.05567818, ...,  0.00313668,
        -0.00453376,  0.07990168],
       [ 0.07479633,  0.00498642,  0.06980991, ...,  0.00534075,
        -0.03365046,  0.05677125],
       ...,
       [ 0.12378179,  0.01707335,  0.05975672, ...,  0.00516062,
         0.02474597, -0.01777882],
       [ 0.10577835,  0.00881486,  0.05729661, ...,  0.00395897,
        -0.07159606, -0.02968549],
       [ 0.10105261,  0.01212631,  0.04042104, ...,  0.00348389,
        -0.01466391,  0.01397732]])